# Programming a logistic regression

This notebook is based on a fabulous [Kaggle tutorial by DATAI](https://www.kaggle.com/kanncaa1/deep-learning-tutorial-for-beginners) and uses the "sign language digits data set", also found through the link.

We start by loading the relevant packages:

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## 1. The dataset

The dataset contains 64x64 images of the signs used to represent the ten digits, 0-9. Indexes 204 to 408 of the dataset show the sign for zero and indexes 822 to 1027 show the sign for one.

In [ ]:
X = np.load('digits_X.npy')
y = np.load('digits_y.npy')

Each value of `X` is a matrix with pixel values, while each value of `y` is a vector representing the value of the digit (one-hot encoded):

In [ ]:
X[204].shape

In [ ]:
y[204]

We can, of course, display the images:

In [ ]:
img_size = 64
plt.subplot(1, 2, 1)
plt.imshow(X[204].reshape(img_size, img_size))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(X[822].reshape(img_size, img_size))
plt.axis('off')

We only need the zeros and ones for our purposes. Hence, start by gathering only the relevant X-variables:

In [ ]:
X = np.concatenate((X[204:409], X[822:1028] ), axis=0)

For the ys, we also only want the relevant ones. Moreover, we want to make sure that instead of a vector, we simply have 0 if the digit is zero and 1 if it is one:

In [ ]:
z = np.zeros(409-204)
o = np.ones(1028-822)
y = np.concatenate((z, o), axis=0).reshape(X.shape[0],1)

With the `reshape`, we make sure that `y` is a vector with two dimensions:

In [ ]:
print(X.shape)
print(y.shape)

Next, we split the data into training and testing with 15% in the test set (you know the drill):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=172)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Finally, we need to "flatten" the Xs. Currently, our input is three-dimensional (each observation is a matrix). However, when we run regressions (or train models more generally), we usually have two-dimensional inputs, as it makes things a lot easier to work with. There are exceptions to this of course, specifically when using convolutional neural networks, but let's not get ahead of ourselves.

What we will do is to convert each matrix (each observation's X-value) to a vector, simply by stacking all the columns of the matrix. If $X^{(i)} \in \mathcal{R}^{n \times m}$, then the fitting vector $\hat{X}^{(i)} \in \mathcal{R}^{n m}$. So we reshape accordingly:

In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test_flat = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
print(X_train_flat.shape)
print(X_train_flat.shape)

We have 4096 pixels per observation, neatly stacked in a vector. All observations together (349 for train, 62 for test), gives us a (two-dimensional) matrix.

## 2. Logistic regression

As you know, we are going to create a model $\hat{y}^{(i)} = \sigma(X^{(i)} w + b) = \sigma(X_1^{(i)}  w_1  + X_2^{(i)} w_2  + ... + X_{4096}^{(i)} w_{4096}  + b)$. "Training" the model, means that we are finding the "right" $w_j$'s and $b$. The right ones are those that minimize our training loss (we'll turn back to that in a bit).

### Initialization

We have to start somewhere. In particular, we are going to initialize our 4097 parameters first. For the 4096 $w_j$'s, we use small random values. The variable $b$ ("bias") will be initialized to 0. We will discuss later in more detail the correct strategy for initialization.

To do this systematically, we define a function to call.
Can you complete this function using `np.random.randn(dimension,1)*0.01`? The function should return `w` (a vector) and `b` (a scalar).

In [ ]:
def initialize_parameters(seed=392,dimension=4096):
    np.random.seed(seed)
    w = 0.01*np.random.randn(dimension,1)
    b = 0
    return w, b

A quick try:

In [ ]:
w, b = initialize_parameters(seed=123,dimension=4096)
print(w)
print(b)

### Forward propagation

When training a neural network, in each step, we start with computing the cost that we aim to minimize, given our current choice of parameters $w$ and $b$. This cost is made up of the "losses" on each training observation. In particular, we have the loss on observation $i$ that is
$$
L^{(i)} = -y^{(i)} \log \hat y^{(i)} - (1-y^{(i)}) (1-\log \hat y^{(i)})
$$
and that sums up to our cost
$$
J = \frac{1}{n}\sum_{i=1}^n L^{(i)}.
$$
Remember also, that we had $\hat{y}^{(i)} = \sigma(X^{(i)} w + b)$.

This step is called "forward propagation" (the reason is easy to see if you look at the computation graph).

Before we actually implement our forward propagation function, however, we have to discuss computation speed. We could, in principal, consider every observation individually in a for-loop, and for each observation, we can consider each pixel in an inner for-loop. However, the following example will show you that this is quite slow compared to the "vectorized" implementation we can get in `numpy`:

In [ ]:
# Running things with two for-loops:
tic = time.process_time()
cost = 0
for i in range(X_train_flat.shape[0]):
    z = b
    for j in range(X_train_flat.shape[1]):
        z += w[j]*X_train_flat[i,j]
    yHat = 1/(1 + np.exp(-z))
    cost += -y_train[i]*np.log(yHat) - (1-y_train[i])*np.log(1-yHat)
    cost /= X_train_flat.shape[0]
toc = time.process_time()
print ("For-loops: Cost = " + str(cost) + ", computation time = " + str(1000 * (toc - tic)) + "ms")

# Running things "vectorized":
tic = time.process_time()
z = np.dot(X_train_flat,w) + b # Can you see why we do np.dot this way round?
yHat = 1/(1 + np.exp(-z))
cost = np.sum(-y_train*np.log(yHat) - (1-y_train)*np.log(1-yHat)) / X_train_flat.shape[0]
toc = time.process_time()
print ("Vectorized: Cost = " + str(cost) + ", computation time = " + str(1000 * (toc - tic)) + "ms")

From this, it should be clear that we want to always vectorize our computations as much as possible! Let's now use the above to implement a forward propagation function.

Can you complete the function and return both the `cost`, as well as the predicted `yHat`? (Hint: all the relevant parts are already in the computation speed comparison and only require minimal changes)

In [ ]:
def forward_propagation(w,b,X,y):
    z = np.dot(X,w) + b # Can you see why we do np.dot this way round?
    yHat = 1/(1 + np.exp(-z))
    cost = np.sum(-y*np.log(yHat) - (1-y)*np.log(1-yHat))/X.shape[0]
    return cost, yHat

Note that we are not just storing the loss, but also the $\hat y$ we computed. We will need those later!

### Back-propagation

Now that we have found the cost, we want to minize that. In order to do so, we use gradient descent. But for that, we first need the derivatives of the cost to our $w$'s and to $b$. Remember that computing derivatives from a computation graph, we start at the end and move backward. Hence the name "back-propagation". Going through all the steps, we arrive at the following derivatives:
$$ \frac{\partial J}{\partial w_j} = \frac{1}{n} \sum_{i=1}^n (  \hat y^{(i)} - y^{(i)} ) X^{(i)}_j,$$
$$\nabla_{w} J = \frac{1}{n} X^T (\hat y - y ),$$
$$\nabla_{b} J = \frac{\partial J}{\partial b} = \frac{1}{n} \sum_{i=1}^n (  \hat y^{(i)} - y^{(i)} ).$$

**Exercise: Verify (using the computation graph) that the derivatives are correct**.

Can you complete the function below, using the formulas here? Make sure to return a vector `dw` with all the $\frac{\partial J}{\partial w_j}$'s (in the same shape as the vector `w`!) and a scalar `db` with $\frac{\partial J}{\partial b}$.

In [ ]:
def back_propagation(w,b,X,y,yHat):
    dw = np.dot(X.T,yHat-y)/X.shape[0]
    db = np.sum(yHat - y)/X.shape[0]
    return dw, db

### Parameter update

In each iteration of our learning procedure, we update the parameters, hopefully moving closer towards the optimum. How we update the parameters is determined by the gradient, as well as the learning rate (a hyper-parameter). Let's define one update step:
1. Compute the `forward_propagation` step (returning `cost` and `yHat`)
1. Compute the `back_propagation` step (using `yHat` from `forward_propagation`)
1. Update `w` as follows: $w_j := w_j - \alpha \frac{\partial J}{\partial w_j}$ (remember that, thanks to vectorization, we can avoid for-loops. Also note that $\alpha$ is the learning rate)
1. Update `b` as follows: $b := b - \alpha \frac{\partial J}{\partial b}$
1. Return `w`, `b`, and `cost`

In [ ]:
def parameter_update(w,b,X,y,learning_rate):
    cost, yHat = forward_propagation(w,b,X,y)
    dw, db = back_propagation(w,b,X,y,yHat)
    w = w - learning_rate * dw
    b = b - learning_rate * db
    return w, b, cost

With this in hand, we can now train our model. We will use one final function that takes as input the training data, as well as the hyper-parameters `learning_rate` and `iterations`:

In [ ]:
def model_training(X,y,learning_rate=0.005,iterations=300):
    w, b = initialize_parameters(seed=392,dimension=X.shape[1]) # start by initializing the parameters w, b
    cost_list = []
    for it in range(iterations):
        w,b,cost = parameter_update(w,b,X,y,learning_rate) # for each iteration, update the parameters using forward and back propagation
        cost_list.append(cost)
    return w, b, cost_list

Let's get training. We will train the model for 300 iterations on the (flattened) training data, then take a look at the cost (also called "training loss")

In [ ]:
w,b,cost_list = model_training(X_train_flat,y_train,learning_rate=0.005,iterations=300)

In [ ]:
plt.plot(range(len(cost_list)),cost_list)
plt.show()

Finally, we can use our model to make predictions on the test set. We define another function to do so:

In [ ]:
def predict(w,b,X):
    z = np.dot(X,w) + b
    yHat = 1/(1 + np.exp(-z))
    # if yHat is bigger than 0.5, our prediction is sign one
    # if yHat is smaller than 0.5, our prediction is sign zero
    y_prediction = (yHat > 0.5)
    return y_prediction

Let's now get predictions. Using these predictions, we can evaluate the accuracy of the model (both on the training and on the test set)

In [ ]:
y_prediction_test = predict(w,b,X_test_flat)
y_prediction_train = predict(w,b,X_train_flat)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))

Not too shabby for a very simple classification algorithm, right? Let's take a look at a few of the examples where things where **not** classified correctly:

In [ ]:
print("Misclassified zero sign as 1: " + str(np.where(y_prediction_test - y_test == 1)[0]))
print("Misclassified one sign as 0: " + str(np.where(y_prediction_test - y_test == -1)[0]))

In [ ]:
plt.imshow(X_test[34])

In [ ]:
plt.imshow(X_test[12])